## I. Khai báo thư viện + GPU

## 1. Khai báo thư viện

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

# common libraty
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# model library
import tensorflow as tf
import scipy.integrate as integrate
from tensorflow.python.keras import backend as K
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

base_path = 'gdrive/MyDrive/Incos/DuDoanGiaCaoSu/Data/'

Mounted at /content/gdrive


## 2. Khai báo GPU

In [4]:
# adjust values to your needs
config = tf.compat.v1.ConfigProto(device_count = {'GPU': 1})
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)

# II. Function

In [5]:
# Tách data chuỗi thời gian về data table
def split_value(df,posotion,num_back):  # df: data; posotion: vị trí cần lùi; num_back: số bước lùi
  row = pd.DataFrame(data=df[posotion-num_back:posotion].close_price.values).T
  row.columns =[f'x(t-{num_back-i-1})' for i in np.arange(0,num_back)]
  row = row.rename(columns={'x(t-0)': 'x(t)'})
  row['date'] = df.date[posotion-1]
  return row


# Đọc data từ file excel lên
def _read_data(time, name_stock, num_back):  # time: w-week, m-month, 6m-6month; name_stock: tên loại cổ phiếu; num_back: số bước lùi
  if time =='w':
    df = pd.read_excel(f'{base_path}/SHFE_week_month.xlsx',sheet_name = f'Fri{name_stock}')[['Date','CLOSEPRICE']]
  elif time =='m':
    df = pd.read_excel(f'{base_path}/SHFE_week_month.xlsx',sheet_name = f'EOM{name_stock}')[['Date','CLOSEPRICE']]
  elif time =='6m':
    df_m = pd.read_excel(f'{base_path}/SHFE_week_month.xlsx',sheet_name = f'EOM{name_stock}')[['Date','CLOSEPRICE']]
    list_date_6m = []
    for i in np.arange(-1,-len(df_m),-6): 
      list_date_6m.append(df_m.Date.values[i])
    df = df_m[df_m['Date'].isin(list_date_6m)].copy()
  else:
    print('Không tồn tại data bạn đã đọc')
  df.columns = ['date','close_price']
  df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
  df = df.sort_values(by='date',ascending=True).reset_index(drop=True)

  # Function format data about data table
  df_table = pd.DataFrame()
  for i in np.arange(num_back,len(df)+1,1):
    row_append = split_value(df,i,num_back)
    df_table = df_table.append(row_append)
  df_table = df_table.reset_index(drop=True)
  return df_table


# Tach data train, validation, test
def _split_data(df_table): # df_table: data table

  # Khai báo giá trị đầu vào và giá trị đầu ra
  x = df_table[df_table.columns[:-2]].values
  y = df_table[df_table.columns[-2:-1]].values
  # Tách data train, validation, test

  X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
  return  X_train, y_train, X_val, y_val, X_test, y_test


# Xây dựng model Random Forest
def _build_model(x_train, y_train, num_tree, max_deep): # x_train: giá trị đầu vào; y_train: giá trị đầu ra; num_tree: số cây, max_deep: chiều sâu cây
  model = RandomForestRegressor(n_estimators=num_tree, max_depth=max_deep, random_state=42)
  model.fit(x_train, y_train.ravel())
  return model

# Dự đoán:
def _score_predict(x_train_val, y_train_val, x_test, y_test, best_num_tree, best_max_deep):  # x_train_val, y_train_val: giá trị đầu vào/đầu ra để train; x_test: giá trị đầu vào cần dự đoán, y_test: giá trị thực tế, num_tree: số cây, max_deep: chiều sâu cây
  predict_price = []
  for i in np.arange(0,len(x_test)):
    model = _build_model(x_train_val, y_train_val, best_num_tree, best_max_deep)
    predict_test = model.predict(x_test[[i]])[0]
    predict_price.append(predict_test)
    x_train_val = np.concatenate([x_train_val, x_test[[i]]])
    y_train_val = np.concatenate([y_train_val, y_test[[i]]])

  APE=100*(abs((y_test.ravel()-predict_price)/y_test.ravel()))
  return np.mean(APE)

# III. Chạy tìm best params cho model

## 1. Khai báo biến



In [23]:
# node_input_s = [1, 2, 3, 4, 5, 8, 10, 12]
node_input_s = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,13,14,15]
name_data = 'MaxVol'   # 1, 3, 4, 5, 6, 7, 8, 9, 10, 11 , MaxVol
name_time = '6m'  # w, m, 6m

# 2. Chạy

In [24]:
for name_data in ['MaxVol']:
  df_result  = pd.DataFrame()
  for n_input in node_input_s:
    data_table = _read_data(name_time, name_data, n_input+1)

    x_train, y_train, x_val, y_val, x_test, y_test = _split_data(data_table)

    score_s = []
    deep_s = []
    num_tree_s = []

    for j in [2,4,5,10,15,20,25,50,100,200]: # Số lượng cây
      for i in np.arange(2,10): # Chiều sâu cây từ 2 đến 10
        model = _build_model(x_train, y_train, j, i)
        score_r = model.score(x_val, y_val)
        score_s.append(round(score_r,2))
        deep_s.append(i)
        num_tree_s.append(j)


    dict_data = {'deep':deep_s,'num_tree':num_tree_s, 'score':score_s}
    results_x = pd.DataFrame(dict_data)
    results_x = results_x.pivot('deep','num_tree','score')

    results_x['max'] = results_x.max(axis=1)
    results_x['index_max'] = results_x.apply(lambda x: np.argmax(x), axis=1)

    index_max_num_tree =  np.argmax(results_x['max'])

    number_of_tree = results_x.columns[results_x['index_max'][results_x.index[index_max_num_tree]]]
    number_of_deep = results_x.index[index_max_num_tree]

    mape = _score_predict(x_train, y_train, x_val, y_val, number_of_tree, number_of_deep) 


    row = {'NodeInput':n_input, 'Tree':number_of_tree, 'Deep': number_of_deep, 'MAPE' :mape }

    df_result = df_result.append(row, ignore_index=True)
    print('Done: ', n_input)
    
  df_result = df_result.sort_values(by='MAPE', ascending=True)
  print(df_result.head(2))
  df_result.to_excel(f'{base_path}/rd_model_history/{name_time}/data_results_{name_data}.xlsx')

Done:  1
Done:  2
Done:  3
Done:  4
Done:  5
Done:  6
Done:  7
Done:  8
Done:  9
Done:  10
Done:  11
Done:  12
Done:  13
Done:  14
Done:  15
    NodeInput  Tree  Deep      MAPE
14       15.0   2.0   2.0  3.842665
4         5.0   2.0   4.0  9.538729


# IV. Dự đoán và tính sai số trên tập test

In [ ]:
name_time = '6m'
name = 'MaxVol'
n_input = 15
num_tree = 2
max_deep = 2

df_table = _read_data(name_time, name, n_input+1)
x_train, y_train, x_val, y_val, x_test, y_test = _split_data(df_table)
mape = _score_predict(np.concatenate([x_train, x_val]), np.concatenate([y_train, y_val]), x_test, y_test, num_tree, max_deep) 
mape

21.833341156374498

# V. Phân loại mức độ dao động

In [ ]:
# Dùng để tính xác xuất cho sai số:
class ProbaError:
  def __init__(self, 
              price_actual_yesterday=97.555, # giá ngày hôm trước
              price_predict_today=98.555, # giá dự đoán từ model cho ngày hôm nay
              params_normal=(0.2, 0.005),  # biến trong phân phối chuẩn (trung bình, độ lệch chuẩn)
              range_change_rate=[-0.05, 0.03]):
    
  # assign params
      self.price_actual_yesterday = price_actual_yesterday
      self.price_predict_today = price_predict_today
      self.mean_error = params_normal[0]
      self.sigma_error = params_normal[1]
      self.lower_change_rate = range_change_rate[0]
      self.upper_change_rate = range_change_rate[1]

# Phân phối chuẩn
  def _buil_normal(self, x, mu, sigma): # mu: trung bình, sigma: độ lệch chuẩn
      return np.exp(-(x-mu)**2 / (2*sigma**2))/(sigma*np.sqrt(2*np.pi))

# Tìm khoảng sai số: (price_predict_today) with (price_actual_yesterday in range_change_rate)
  def _find_range_error(self, price_actual_yesterday, price_predict_today, lower_change_rate, upper_change_rate): # lower_change_rate: cận dưới, upper_change_rate: cận trên
  # interval price
      thresold_price_today_1 = price_actual_yesterday * (1 + lower_change_rate)
      thresold_price_today_2 = price_actual_yesterday * (1 + upper_change_rate)

  # interval error:
      thresold_error_today_1 = price_predict_today - thresold_price_today_1
      thresold_error_today_2 = price_predict_today - thresold_price_today_2

      if thresold_error_today_1 < thresold_error_today_2:
          return thresold_error_today_1, thresold_error_today_2

      return thresold_error_today_2, thresold_error_today_1


#  Hàm mật độ của phân phối chuẩn
  def density_proba_normal(self):
  
      lower_error_today, upper_error_today = self._find_range_error(self.price_actual_yesterday, self.price_predict_today, self.lower_change_rate, self.upper_change_rate)
      val, err = integrate.quad(lambda x : self._buil_normal(x, self.mean_error, self.sigma_error),
                                lower_error_today , upper_error_today)
      return round(val,3)

In [ ]:
# Dùng để tính xác xuất dao động của ngày hôm nay so với ngày hôm trước:
class PredictPrice:
  def __init__(self,
               df, 
               num_tree, 
               max_deep,
              description_model="Dự báo tuần và sai số trên tập test là 2.8%"
            ):
    # params
    self.df = df
    self.num_tree = num_tree
    self.max_deep = max_deep
    self.description_model = description_model


    # Split feature label and label variables:
    feature_label_name = df.columns[:-2]
    label_name = df.columns[-2:-1]

    self.x=df[feature_label_name].values
    self.y=df[label_name].values.ravel()

  def _fit_model_rf(self):
    rf_model = RandomForestRegressor(n_estimators=self.num_tree, max_depth=self.max_deep, random_state=42)
    rf_model.fit(self.x[:-1], self.y[:-1])
    return rf_model

  # get params for error distribution
  def get_params_error(self):
    
    predict_price_s = self._fit_model_rf().predict(self.x[:-1])

    # Scaling the y_test Price data back to original price scale
    truth_price_s = self.y[:-1]

    # calculate errors
    error_s = predict_price_s - truth_price_s

    # get params error_s
    mean_error = np.mean(error_s)
    sigma_error = np.sqrt(sum((error_s - mean_error)**2)/len(error_s))

    # return
    return mean_error, sigma_error

  # get pair price
  def get_pair_price(self):
      price_actual_yesterday = self.y[-2:-1]
      price_predict_today = self._fit_model_rf().predict(self.x[-1:])

      return price_actual_yesterday, price_predict_today

  # run class
  def run(self):
      mean_error, sigma_error = self.get_params_error()
      price_actual_yesterday, price_predict_today = self.get_pair_price()

      dict_range_error_s = {
            'Nhỏ hơn -5%': [-np.inf, -0.05],
            'Từ -5% đến -3%': [-0.05, -0.03],
            'Từ -3% đến 0%': [-0.03, 0],
            'Từ 0% đến 3%': [0, 0.03],
            'Từ 3% đến 5%': [0.03, 0.05],
            'Lớn hơn 5%': [0.05, np.inf]
            }

      result = {
          'Ghi chú': self.description_model
          }
      for description, range_change_rate in dict_range_error_s.items():
          density_proba = ProbaError(price_actual_yesterday,
                                    price_predict_today,
                                    params_normal=(mean_error, sigma_error), 
                                    range_change_rate=range_change_rate).density_proba_normal()
          row = {description: round(density_proba*100,2)}
          result.update(row)

          df_result = pd.DataFrame(result, index=[0])
          df_result['Date'] = self.df.date.values[-1]

      return df_result

In [ ]:
classifi_df = pd.DataFrame()
for i in range(-10,0,1):
    split_data = df_table[:i]
    classifi_point = PredictPrice(split_data, 
                                  num_tree = num_tree, 
                                  max_deep = max_deep,
                                  description_model="Dự báo tuần và sai số trên tập test là 8%").run()
    classifi_df = classifi_df.append(classifi_point)


# Predict max point 
classifi_point = PredictPrice(df_table, 
                              num_tree = 10, 
                              max_deep = 6,
                              description_model="Dự báo tuần và sai số trên tập test là 2.8%").run()
classifi_df = classifi_df.append(classifi_point)

# Classification trend
classifi_df['Increase'] = classifi_df[['Từ 0% đến 3%', 'Từ 3% đến 5%', 'Lớn hơn 5%',]].sum(axis=1)
classifi_df['Decrease'] = classifi_df[['Nhỏ hơn -5%', 'Từ -5% đến -3%', 'Từ -3% đến 0%']].sum(axis=1)

In [ ]:
classifi_df[['Date', 'Nhỏ hơn -5%', 'Từ -5% đến -3%', 'Từ -3% đến 0%','Từ 0% đến 3%', 'Từ 3% đến 5%', 'Lớn hơn 5%','Increase','Decrease']]

,Date,Nhỏ hơn -5%,Từ -5% đến -3%,Từ -3% đến 0%,Từ 0% đến 3%,Từ 3% đến 5%,Lớn hơn 5%,Increase,Decrease
0,2018-02-28,77.6,1.8,2.5,2.3,1.4,14.4,18.1,81.9
0,2018-08-31,38.4,1.7,2.7,2.7,1.8,52.7,57.2,42.8
0,2019-02-28,35.3,1.7,2.6,2.6,1.8,56.1,60.5,39.6
0,2019-08-30,58.1,1.7,2.5,2.5,1.6,33.6,37.7,62.3
0,2020-02-28,43.6,1.8,2.6,2.7,1.8,47.5,52.0,48.0
0,2020-08-31,43.0,1.8,2.7,2.7,1.8,48.1,52.6,47.5
0,2021-02-26,51.0,2.0,3.0,3.0,1.9,39.1,44.0,56.0
0,2021-08-31,5.5,0.8,1.3,1.5,1.1,89.7,92.3,7.6
0,2022-02-28,45.4,3.1,4.6,4.6,3.0,39.4,47.0,53.1
0,2022-08-31,42.1,3.0,4.6,4.6,3.0,42.6,50.2,49.7
